In [2]:
!pip install pandas-datareader
import pandas_datareader.data as web
import pandas as pd
import datetime as dt



# Mimicry as measure of market panic

https://www.researchgate.net/publication/48207581_Predicting_Economic_Market_Crises_Using_Measures_of_Collective_Panic

The mimicry index is based on research into collective panic, the original research used the russel 3000, herein an analysis of the S&P 500.

## Import S&P 500 Data or download from Yahoo Finance
Sticking to adjusted daily close data

In [113]:
def loadSp500(measured='Adj Close', fromFile=True):
    """Reads data for the S&P 500"""
    if fromFile:
        return pd.read_csv('SP500Close.csv', parse_dates=True).set_index('Date')
    else:
        spList = pd.read_csv("sp-500.csv").Symbol.values[:-1]
        df = web.DataReader(spList, 'yahoo')
        df = df.T.reset_index()
        df = df[df.Attributes=='Adj Close'].iloc[:,3:].T
        df.columns = spList
        df.to_csv('SP500Close.csv')
        return df
df = loadSp500(measured='Adj Close')
# df.T.dropna()

# The murmurs function 

The murmurs function  measures the share of the market moving in the same direction on a scale from 0 to 1. Mimicry is measured independently of direction of movement.

In [154]:
def murmurs(df, percentile=True):
    """Murmurations are measured on a scale from 0 to 1 with 1 being perfectly homogenous behaviour and 0 max entropy"""
    import numpy as np
    nas = df.isna()
    df = df.pct_change(1)
    df = df > 0
    df[nas] = np.nan
    # print(df)
    df.sum(axis=0)
    murmuration=pd.DataFrame(((df.mean(axis=1) - .5 )*2), columns=['MIX']).abs().reset_index()
    murmuration['year']=pd.to_datetime(murmuration.Date).dt.year
    murmuration['MIX_3dma'] = murmuration['MIX'].rolling(window=3).mean()
    murmuration['MIX_20dma'] = murmuration['MIX'].rolling(window=20).mean()

    if percentile:
        murmuration[['MIX','3dma','20dma']]=murmuration[['MIX','3dma','20dma']].rank(pct=True)
    murmuration.set_index('Date')
    return murmuration
murmuration = murmurs(df,percentile=False)



In [155]:
import plotly.express as px
from datetime import datetime as dt
import plotly.graph_objects as go

fig=px.line(murmuration, x='Date',y=['MIX_20dma'],width=1000)
fig.add_trace(go.Scatter(x=murmuration.Date, y=murmuration['MIX'],                    mode='markers',
                    name='MIX'))
spy=web.DataReader('spy', 'yahoo')['Adj Close']
spx=spy/spy[0]
# fig.add_trace(go.Scatter(x=month, y=low_2014, name = 'Low 2014',
#                          line=dict(color='royalblue', width=4)))                    
fig.add_trace(go.Scatter(y=spx.values,x=spx.index,name="SPY"))
# fig.add_hline(0.5)
fig.update_traces(marker_size=3)
# fig.add_vline(x=dt(2018,12,21))
# fig.add_vline(x=dt(2018,4,2))

# fig.add_vline(x=dt(2019,6,3))
fig.add_vline(x=dt(2020,3,23),line_width=2, line_dash="dash", line_color="green")
# ,annotation_text="23 Mar 2020", annotation_position="top left")
fig.add_hrect(y0=0.8, y1=1.05, line_width=0, fillcolor="red", opacity=0.2)
# fig.add_vline(x=dt(2020,9,24))
# fig.add_vline(x=dt(2020,10,30))
# fig.add_vline(x=dt(2022,6,16))
fig.add_annotation(x=dt(2020,3,23), y=2.0,
            text="23 Mar 2020",
            showarrow=False,
            xshift=40,
            yshift=-10,
            )

fig.update_layout(title='Mimicry Index SP500 (MIX)', title_x=0.5)
fig